In [1]:
! pip install film_review_scraper

In [3]:
FILM_NAME = "The_Last_Tycoon"

IMDB_URL = "https://www.imdb.com/title/tt0074777/reviews?sort=submissionDate&dir=desc&ratingFilter=0"
RT_URL = "https://www.rottentomatoes.com/m/last_tycoon/reviews?type=user"
DB_SHORT_URL = "https://movie.douban.com/subject/1298293/comments?sort=time&status=P"
DB_LONG_URL = "https://movie.douban.com/subject/1298293/reviews?sort=time"

FILM_DATA_FILE = "/Users/pipchang/Documents/VSC/Projects/DH-S/the_last_tycoon/the_last_tycoon.yaml"
OUTPUT_FOLDER = "/Users/pipchang/Documents/VSC/Projects/DH-S/the_last_tycoon"

In [ ]:
# for IMDB and Rotten Tomatoes
from film_review_scraper import IMDB, get_output_path, save_dataclass_to_jsonl
website = IMDB()
review_blocks = website.fetch_reviews(url=IMDB_URL)
reviews = website.parse_reviews(review_blocks)
output_path = get_output_path(
    folder_path=OUTPUT_FOLDER,
    file_name=f"imdb_{FILM_NAME}_newest",  # can change according to liking
    file_type="jsonl",
)
save_dataclass_to_jsonl(objects=reviews, output_path=output_path)


In [ ]:
# for Douban
from film_review_scraper import Douban, get_output_path, save_dataclass_to_jsonl
website = Douban()
review_blocks = website.fetch_reviews(
    url="https://movie.douban.com/subject/35209731/comments?sort=time&status=P",
    review_type="short",  # change accordingly
)
reviews = website.parse_reviews(review_blocks=review_blocks, parse_type="short") # change accordingly
output_path = get_output_path(
    folder_path=OUTPUT_FOLDER,
    file_name=f"db_{FILM_NAME}_short_popular",  # can change according to liking
    file_type="jsonl",
)
save_dataclass_to_jsonl(objects=reviews, output_path=output_path)

In [3]:
# merge datas
from film_review_scraper import get_output_path, FilmData, ReviewData, merge_film_and_review_data, save_dicts_to_jsonl

INPUT_FOLDER = "/Users/pipchang/Documents/VSC/Projects/DH-S/download/Top_Gun_Maverick"
OUTPUT_FOLDER = "/Users/pipchang/Documents/VSC/Projects/DH-S/download/Top_Gun_Maverick/Merged"
FILM_DATA_FILE = ""
FILE_NAME = "Top_Gun_Maverick_Merged"

# film_data = FilmData.from_file(FILM_DATA_FILE)
review_data = ReviewData.from_folder(
    folder_path=INPUT_FOLDER, film_name="Top Gun Maverick"
)
# all_data = merge_film_and_review_data(film_data=film_data, review_data=review_data)
output_path = get_output_path(
    folder_path=OUTPUT_FOLDER, 
    file_name=FILE_NAME, 
    file_type="jsonl"
)
save_dicts_to_jsonl(list_of_dicts=review_data.data, output_path=output_path)